<a href="https://colab.research.google.com/github/dudeh534/Mirae_Exit/blob/main/sector_input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#구글드라이브 연동용
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
#os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)

os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

Mounted at /content/drive


In [ ]:
import time
import json
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import oracledb
import re

#날짜가져오기
url = 'https://finance.naver.com/sise/sise_deposit.nhn'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one(
    'div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)

sector_code = [
    'G25', 'G35', 'G50', 'G40', 'G10', 'G20', 'G55', 'G30', 'G15', 'G45'
]

data_sector = []

for i in tqdm(sector_code):
    url = f'''http://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt={biz_day}&sec_cd={i}'''
    data = rq.get(url).json()
    data_pd = pd.json_normalize(data['list'])

    data_sector.append(data_pd)

    time.sleep(2)

kor_sector = pd.concat(data_sector, axis = 0)
kor_sector = kor_sector[['IDX_CD', 'CMP_CD', 'CMP_KOR', 'SEC_NM_KOR']]
kor_sector['기준일'] = biz_day
kor_sector['기준일'] = pd.to_datetime(kor_sector['기준일'])

#오라클 연결
connection=oracledb.connect(
     user="admin",
     password='rlaTPWNS!123',
     dsn="gangnamdatabase_high",
     config_dir="/content/drive/MyDrive/Colab Notebooks",
     wallet_location="/content/drive/MyDrive/Colab Notebooks",
     wallet_password='rlaTPWNS!123')

mycursor = connection.cursor()
sql = '''INSERT INTO kor_ticker (기준일, 종목코드, 종목명, 시장구분, 종가, 시가총액, EPS, 선행EPS, BPS, 주당배당금, 종목구분) VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11)'''
query = '''
    insert into kor_sector (IDX_CD, CMP_CD, CMP_KOR, SEC_NM_KOR, 기준일)
    values (:1,:2,:3,:4,:5)
'''

args = kor_sector.values.tolist()

mycursor.executemany(query, args, batcherrors = False)
connection.commit()

connection.close()

20231114


100%|██████████| 10/10 [00:38<00:00,  3.84s/it]
